In [44]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
from torchtune.modules import RMSNorm
from tokenizers import Tokenizer
from pathlib import Path

In [45]:
!nvidia-smi

Sat Jul  6 13:33:20 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.58.02              Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8              1W /   80W |    2586MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [70]:

@dataclass
class ModelArgs:
    #Hyperparameters

    block_size = 128
    batch_size = 16
    embeddings_dims = 256
    attn_dropout = 0.1
    no_of_heads = 4 #IMP needs to be thoroughly calculated
    dropout = 0.1
    epochs = 100
    max_lr = 1e-4
    no_of_decoder_layers = 6 #IMP needs to be thoroughly calculated
    weight_decay_optim = 0.1
    beta_1 = 0.9
    beta_2 = 0.95
    clip = 1.0
    device = 'cuda'
    no_kv_heads = 2
    vocab_size = 10000

In [174]:
#Collab setup
data_path = Path('data')
data_path.mkdir(exist_ok=True)
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
!cp input.txt data/input.txt


--2024-07-06 12:13:32--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-07-06 12:13:33 (17.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [71]:
#Datasets

# Using tinyshakespeare

with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [72]:

#Subword level tokenization

#Loading custom trained BPE
# Load the tokenizer
tokenizer = Tokenizer.from_file("data/bpe_tokenizer_tinyshakespeare_1k.json")
vocab_size = tokenizer.get_vocab_size()
# Encode and decode functions
encode = lambda s: tokenizer.encode(s).ids
decode = lambda l: tokenizer.decode(l)

In [73]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - ModelArgs.block_size, (ModelArgs.batch_size,))
    x = torch.stack([data[i:i+ModelArgs.block_size] for i in ix])
    y = torch.stack([data[i+1:i+ModelArgs.block_size+1] for i in ix])
    x, y = x.to(ModelArgs.device), y.to(ModelArgs.device)
    return x, y

In [74]:
class Normalization(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims
    ):  
        super().__init__()
        self.rmsnorm_layer = RMSNorm(dim=embeddings_dims)
        
        
    def forward(self, x):
        
        x = self.rmsnorm_layer(x)
        return x
        

In [75]:
import numpy as np
class RotaryEmbeddings(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        block_size: int = ModelArgs.block_size,
        batch_size: int = ModelArgs.batch_size
    ):
        super().__init__()
        
        self.embeddings_dims = embeddings_dims
        self.block_size = block_size
        self.batch_size = batch_size
        self.theta = 0  
        # self.init_matrix(self.block_size)
        
        # print("MATRXO: ", self.rotatory_matrix)
        
    def init_matrix(self, seq_len):
            self.matrix = torch.zeros((seq_len, self.embeddings_dims, self.embeddings_dims), dtype=torch.float32, device=ModelArgs.device, requires_grad=False)
            for pos in range(seq_len):
                for j in range(1, self.embeddings_dims // 2):
                    self.theta = 10000 ** (-2*(pos-1) / self.embeddings_dims)
                    self.matrix[pos, 2*j + 1, 2*j + 1] = np.cos((pos*self.theta))
                    self.matrix[pos, 2*j + 1, j + 1] = -np.sin((pos* self.theta))
                    self.matrix[pos, 2*j , 2*j ] = -np.cos((pos* self.theta))
                    self.matrix[pos, 2*j + 1, 2*j + 1] = np.sin((pos* self.theta))
            return self.matrix
        
    def forward(self, x):
        # B,T,C = x.shape
        # print("MATRIX:",x)
        if(x > self.block_size):
            matrix = self.init_matrix(x)
            return matrix
        else:
            matrix = self.init_matrix(self.block_size)
            
            return matrix

In [127]:
rot = RotaryEmbeddings()
res = rot(128)
res.shape

MATRIX: 128


torch.Size([128, 256, 256])

In [76]:
class RotaryAttentionHead(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        no_of_heads: int = ModelArgs.no_of_heads,
        attn_dropout: int = ModelArgs.attn_dropout
    ):
        super().__init__()
        self.head_size = embeddings_dims // no_of_heads
        self.query = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False, dtype=torch.float32)
        self.key = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False, dtype=torch.float32)
        self.value = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False, dtype=torch.float32)
        self.rotary_matrix = RotaryEmbeddings(embeddings_dims=embeddings_dims)
        self.dropout = nn.Dropout(p = attn_dropout)
        
    def forward(self,x):
        # print(x.shape)
        batch, block_size, embeddings_dims = x.shape
        query = self.query(x)
        # print(query)
        key = self.key(x)
        values = self.value(x)
        matrix = self.rotary_matrix(block_size)
        
        # print(matrix.shape)
        # print(query.shape)
        masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
        rotary_query = matrix @ query.permute(1,2,0) # (B,T, C,C) @ (B,T,C) -> (B,C,T) = (B,T,C,T)
        rotary_key = matrix @ key.permute(1,2,0)  #  (B,T, C,C  ) @ (B,T,C) -> (B,C,T) = (B,T,C,T)
        weights = rotary_query.permute(2,0,1) @ rotary_key.permute(2,0,1).transpose(-2, -1)#(B,T,C,T) @ (B,T,C,T) = (T,C,C,T)
        weights_masked = weights.masked_fill(masked == 0, float('-inf'))
        scaled_weights = weights_masked / (torch.sqrt(torch.tensor(key.shape[-1])))
        scaled_weights = F.softmax(scaled_weights, dim=-1)
        value = scaled_weights @ values
        out = self.dropout(value)
        return out

In [129]:
rh = RotaryAttentionHead()
random_data = torch.randn((ModelArgs.batch_size, ModelArgs.block_size, ModelArgs.embeddings_dims))
res = rh(random_data)
res.shape

MATRIX: 128


torch.Size([16, 128, 256])

In [77]:
class MQA(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        block_size: int = ModelArgs.block_size,
        no_of_kv_heads: int = ModelArgs.no_of_heads,
        no_of_heads: int = ModelArgs.no_of_heads
    ):
        super().__init__()
        
        self.no_of_kv_heads = no_of_kv_heads
        self.no_of_q_heads = no_of_heads // no_of_kv_heads
        self.head_size = embeddings_dims // self.no_of_q_heads
        self.rotary_matrix = RotaryEmbeddings(embeddings_dims=embeddings_dims)
        # self.query = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        self.key = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, dtype=torch.float32, bias=False)
        self.value = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, dtype=torch.float32, bias=False)
        self.dropout = nn.Dropout(p = ModelArgs.attn_dropout)
        self.linear_layer = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, dtype=torch.float32, bias=False)
        
        
        
    def scaled_dot_product(self, q, k, v, block_size, matrix):
            
            masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
            # print("Before: ")
            # print(q.shape)
            # print(torch.transpose(q, dim0=-2, dim1=-1).shape)
            # print(matrix.shape)
            # print(k.shape)
            # print(torch.transpose(k, dim0=-2, dim1=-1).shape)
            # rotary_query = matrix @ torch.transpose(q, dim0=-2, dim1=-1)
            # rotary_key = matrix @ torch.transpose(k, dim0=-2, dim1=-1)
            # print("After: ")
            # print(q.shape)
            # print(matrix.shape)
            # print(k.shape)
            masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
            rotary_query = matrix @ q.permute(1,2,0) # (B,T, C,C) @ (B,T,C) -> (B,C,T) = (B,T,C,T)
            rotary_key = matrix @ k.permute(1,2,0)  #  (B,T, C,C  ) @ (B,T,C) -> (B,C,T) = (B,T,C,T)
            weights = rotary_query.permute(2,0,1) @ rotary_key.permute(2,0,1).transpose(-2, -1)#(B,T,C,T) @ (B,T,C,T) = (T,C,C,T)
            weights_masked = weights.masked_fill(masked == 0, float('-inf'))
            scaled_weights = weights_masked / (torch.sqrt(torch.tensor(k.shape[-1])))
            scaled_weights = F.softmax(scaled_weights, dim=-1)
            value = scaled_weights @ v
            out = self.dropout(value)
            return value
    
    def forward(self,x):
        # print("MQA: ", x.shape)
        batch, block_size, embeddings_dims = x.shape
        multi_query = nn.ModuleList([nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False) for _ in range(self.no_of_q_heads)])
        # query = self.query(x)
        matrix = self.rotary_matrix(block_size)
            

        key = self.key(x)
        values = self.value(x)
        # rotary_query = matrix @ torch.transpose(, dim0=1, dim1=0)
        # rotary_key = matrix @ torch.transpose(key, dim0=1, dim1=0)
        # matrix = self.rotary_matrix(block_size)
        # self.mqa = nn.ModuleList([
           
        # ])
        multi_query_concat = torch.cat([self.scaled_dot_product(query(x), key, values, block_size, matrix) for query in multi_query], dim=-1)
        # linear_layer_query = self.linear_layer(multi_query_concat)
        # masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
        # rotary_query = matrix @ torch.transpose(query, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        # rotary_key = matrix @ torch.transpose(key, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        # print(multi_query_concat.shape)
        # print(key.shape)
        # print(linear_layer_query.shape)
        # weights = linear_layer_query @ (torch.transpose(key, dim0=-2, dim1=-1))
        # weights_masked = weights.masked_fill(masked == 0, float('-inf'))
        # scaled_weights = weights_masked / (key.shape[-1] ** -0.5)
        # scaled_weights = F.softmax(scaled_weights, dim=-1)
        # value = scaled_weights @ values
        
        linear_layer= self.linear_layer(multi_query_concat)
        out = self.dropout(linear_layer)
        return out

In [78]:
class GQA(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        block_size: int = ModelArgs.block_size,
        no_of_q_heads: int = ModelArgs.no_of_heads,
        no_of_kv_heads: int = ModelArgs.no_kv_heads
    ):
        super().__init__()
        
        # self.head_size = embeddings_dims // no_of_q_heads
        # self.query = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        self.no_of_kv_heads = no_of_kv_heads
        self.no_of_q_heads = no_of_q_heads
        # self.key = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        # self.value = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        self.dropout = nn.Dropout(p = ModelArgs.attn_dropout)
        self.linear_layer = nn.Linear(in_features=embeddings_dims * self.no_of_kv_heads, out_features=embeddings_dims , dtype=torch.float32, device=ModelArgs.device, bias=False)
        
    # def scaled_dot_product(self, q, k, v, block_size):
            
    #         masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
    #         weights = q @ (torch.transpose(k, dim0=-2, dim1=-1))
    #         weights_masked = weights.masked_fill(masked == 0, float('-inf'))
    #         scaled_weights = weights_masked / (k.shape[-1] ** -0.5)
    #         scaled_weights = F.softmax(scaled_weights, dim=-1)
    #         value = scaled_weights @ v
    #         return value
        
        
    def forward(self,x):
        
        batch, block_size, embeddings_dims = x.shape
        mqa = nn.ModuleList([MQA(embeddings_dims=embeddings_dims, block_size=block_size) for _ in range(self.no_of_kv_heads)])
        # query = self.query(x)
        # key = self.key(x)
        # values = self.value(x)
        # matrix = self.rotary_matrix(block_size)
        grouped_query_concat = torch.cat([group(x) for group in mqa], dim=-1)
        # linear_layer_query = self.linear_layer(multi_query_concat)
        # masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
        # rotary_query = matrix @ torch.transpose(query, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        # rotary_key = matrix @ torch.transpose(key, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        # print(multi_query_concat.shape)
        # print(key.shape)
        # print(linear_layer_query.shape)
        # print(grouped_query_concat.shape)     
        linear_layer= self.linear_layer(grouped_query_concat)
        out = self.dropout(linear_layer)
        return out

In [132]:

random_data = torch.randn((ModelArgs.batch_size, ModelArgs.block_size, ModelArgs.embeddings_dims))
gqa = GQA()
# input_data = torch.tensor()
res = gqa(random_data)
res.shape

MATRIX: 128
MATRIX: 128


torch.Size([16, 128, 256])

In [133]:
masked = torch.tril(torch.ones((ModelArgs.block_size, ModelArgs.block_size), device=ModelArgs.device, requires_grad=False))
masked.shape

torch.Size([128, 128])

In [13]:
# class KVCache:
#     def __init__(
#         self,
#         embeddings_dims: int =  ModelArgs.embeddings_dims,
#         block_size: int  = ModelArgs.block_size,
#         no_of_decoder_layers: int =ModelArgs.no_of_decoder_layers
#     ):
#         super().__init__()
#         self.head_size = embeddings_dims / no_of_decoder_layers
#         self.k_cache = torch.ones((block_size, embeddings_dims, self.head_size), device=ModelArgs.device, requires_grad=False)
#         self.v_cache = torch.ones((block_size, embeddings_dims, self.head_size), device=ModelArgs.device, requires_grad=False)
#         self.block_size = block_size,
#         self.embeddings_dims = embeddings_dims
#     def update(
#         self,
#         k: torch.tensor,
#         v: torch.tensor
#     ):
#         self.k_cache[:self.block_size, :self.block_size] = k
#         self.v_cache = v
        
#     def get(self):
        

In [79]:
class Swish(nn.Module):
    def __init__(
        self,
        block_size: int = ModelArgs.block_size,
        embeddings_dims: int = ModelArgs.embeddings_dims
    ):
        super().__init__()
        
        self.sig = torch.nn.Sigmoid()
        
        
    def forward(self, x):
        swish = x * self.sig(x)
        
        return swish
         

In [80]:
class SWiGLU(nn.Module):
    def __init__(
        self,
        block_size: int = ModelArgs.block_size,
        embeddings_dims: int = ModelArgs.embeddings_dims
    ):
        super().__init__()
        
        self.swish = Swish(block_size=block_size, embeddings_dims=embeddings_dims)
        # self.gated_layer = nn.Linear(in_features=block_size, out_features=embeddings_dims, device=ModelArgs.device, bias=False)
        self.linear_layer1 = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False, dtype=torch.float32)
        self.linear_layer2 = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False, dtype=torch.float32)
        self.linear_layer3 = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False, dtype=torch.float32)
        # self.gamma = nn.Parameter(torch.ones((block_size, embeddings_dims), device=ModelArgs.device), requires_grad=True)
        # self.beta = nn.Parameter(torch.ones((block_size, embeddings_dims), device=ModelArgs.device), requires_grad=True)
        
        
        
    def forward(self, x):
        swish_res = self.swish(self.linear_layer1(x))
        x_V = self.linear_layer2(x)
        res = torch.mul(swish_res, x_V)
        out = self.linear_layer3(res)
        return out
         

In [15]:
swiglue = SWiGLU()
res = swiglue(random_data)
res.shape

NameError: name 'random_data' is not defined

In [81]:
class FFN(nn.Module):
    def __init__(self,
                  embeddings_dims: int = ModelArgs.embeddings_dims,
                  block_size: int = ModelArgs.block_size,
                  vocab_size: int = ModelArgs.vocab_size,
                   dropout = ModelArgs.dropout
                 
                 ):
        super().__init__()
        
        self.linear_layer = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, dtype=torch.float32)
        self.swiglue = SWiGLU(block_size=block_size, embeddings_dims=embeddings_dims)
        self.dropout = nn.Dropout(p = dropout)
    def forward(self, x):
        
        x = self.swiglue(x)
        x = self.linear_layer(x)
        x = self.dropout(x)
        return x

In [82]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                embeddings_dims: int = ModelArgs.embeddings_dims,
                dropout = ModelArgs.dropout,
                block_size: int = ModelArgs.block_size,
                vocab_size: int = ModelArgs.vocab_size,
                 
                 ) :
        super().__init__()
        
        
        self.feedforward_network = FFN(embeddings_dims=embeddings_dims, block_size=block_size, vocab_size=vocab_size)
        self.gqa = GQA(embeddings_dims=embeddings_dims, block_size=block_size, no_of_kv_heads=ModelArgs.no_kv_heads, no_of_q_heads=ModelArgs.no_of_heads)
        self.norm = Normalization(embeddings_dims=embeddings_dims)
        self.dropout = nn.Dropout(p = dropout)
    def forward(self, x):
        
        x = self.gqa(x + self.norm(x))
        x = self.feedforward_network(x + self.norm(x))
        return x

In [83]:
class Llama(nn.Module):
    def __init__(self, 
                  embeddings_dims: int = ModelArgs.embeddings_dims,
                  no_of_decoder_layers: int = ModelArgs.no_of_decoder_layers,
                  block_size: int = ModelArgs.block_size,
                  vocab_size: int = ModelArgs.vocab_size,
                  dropout = ModelArgs.dropout
                 
                 ) :
        super().__init__()
        
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embeddings_dims, device=ModelArgs.device, dtype=torch.float32)
        self.decoder = nn.Sequential(*[DecoderLayer(embeddings_dims=embeddings_dims, block_size=block_size, vocab_size=vocab_size, dropout=dropout) for _ in range(no_of_decoder_layers)])
        self.linear_layer = nn.Linear(in_features=embeddings_dims, out_features=vocab_size, device=ModelArgs.device, dtype=torch.float32)
        self.dropout = nn.Dropout(p = dropout)
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        x = self.decoder(x)
        out = self.linear_layer(x)
        return out

In [88]:
#Instantiating the model
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
ModelArgs.device = device
model = Llama(embeddings_dims=ModelArgs.embeddings_dims, block_size=ModelArgs.block_size, vocab_size=ModelArgs.vocab_size, dropout=ModelArgs.dropout)
model = model.to(ModelArgs.device)

In [61]:
idx, targets = get_batch('test')
idx.shape

torch.Size([64, 256])

In [22]:
res = model(idx)
res

KeyboardInterrupt: 

In [85]:
#Printing a summary of the architecture
from torchinfo import summary
idx, targets = get_batch('test')
# idx = idx.to(device)
summary(model=model,
        input_data=idx,
        # input_size=(ModelArgs.batch_size, ModelArgs.block_size, ModelArgs.embeddings_dims),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Trainable
Llama (Llama)                                      [16, 128]            [16, 128, 10000]     --                   True
├─Embedding (embeddings)                           [16, 128]            [16, 128, 256]       2,560,000            True
├─Dropout (dropout)                                [16, 128, 256]       [16, 128, 256]       --                   --
├─Sequential (decoder)                             [16, 128, 256]       [16, 128, 256]       --                   True
│    └─DecoderLayer (0)                            [16, 128, 256]       [16, 128, 256]       --                   True
│    │    └─Normalization (norm)                   [16, 128, 256]       [16, 128, 256]       256                  True
│    │    └─GQA (gqa)                              [16, 128, 256]       [16, 128, 256]       131,072              True
│    │    └─Normalization (norm)             

In [89]:
# Optimizer setup and scheduler steup

optimizer = torch.optim.AdamW(weight_decay=ModelArgs.weight_decay_optim, params=model.parameters(), lr=ModelArgs.max_lr, betas=(ModelArgs.beta_1, ModelArgs.beta_2))
# optimizer = torch.optim.Adam(model.parameters(), lr=max_lr, weight_decay=weight_decay_optim)
# initial_iters = 2000
total_steps = 5000
eval_iters = 100
# warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period=2000)
# lr_scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max= total_steps - initial_iters)
# lr_scheduler_linear = torch.optim.lr_scheduler.LinearLR(optimizer=optimizer, total_iters=initial_iters)

# @torch.inference_mode()
# def estimate_loss():
#     out = {}
#     model.eval()
#     for split in ['val']:
#         # losses = torch.zeros(eval_iters)
#         # for k in range(eval_iters):
#         idx, targets = get_batch(split=split)
#         logits = model(idx)
#         batch_size, block_size, embeddings_dims = logits.shape
#         logits = logits.view(batch_size*block_size, embeddings_dims) # Total tokens(words) => batch_size * block_size
#         targets = targets.view(batch_size * block_size)
#         loss = nn.functional.cross_entropy(logits, targets)
#         # losses[k] = loss.item()
#       # out[split] = losses.mean()
#         out[split] = loss.item()
#     model.train()
#     return out
@torch.inference_mode()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            idx, targets = get_batch(split=split)
            logits = model(idx)
            batch_size, block_size, embeddings_dims = logits.shape
            logits = logits.view(batch_size*block_size, embeddings_dims) # Total tokens(words) => batch_size * block_size
            targets = targets.view(batch_size * block_size)
            loss = nn.functional.cross_entropy(logits, targets)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [90]:
#Train the  model
from tqdm import tqdm

model.train()
for step in tqdm(range(total_steps)):

    # every once in a while evaluate the loss on train and val sets
    if (step  % eval_iters == 0 and step != 0) or step == total_steps - 1:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        torch.save(model.state_dict(), 'weights/Llama7M_steps_%d.pth' % (step))

    idx, targets = get_batch(split='train')
    logits = model(idx)
    batch_size, block_size, embeddings_dims = logits.shape
    logits = logits.view(batch_size*block_size, embeddings_dims)
    targets = targets.view(batch_size * block_size)
    loss = nn.functional.cross_entropy(logits, targets)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # print(loss.item())
    # break

    # if step != 0 and (step % eval_iters == 0 or step == total_steps -1) :
    #     loss_values = estimate_loss()
    #     print("Train Loss at {} steps : {}".format(step, loss.item()), "Val Loss at {} steps : {}".format(step, loss_values['val']))

  2%|▏         | 93/5000 [06:35<5:58:25,  4.38s/it]